# <span style="color:green;font-size:1.2em;">OLS with Docker and OpenStack: Cookbook</span>
<span style="color:green;font-size:1.2em;">Warren Read, EBI SPOT</span>  
<span style="color:green;font-size:1.2em;">24th October 2019</span>  
<span>&nbsp;</span>

A step-by-step guide demonstrating how to build and run your own customised, private and secure instance of EMBL/EBI OLS, running within the EBI’s OpenStack IaaS (Infrastructure as a Service) Embassy Cloud.

## Introduction ##

[OLS](https://www.ebi.ac.uk/ols/index "Public release of EMBL-EBI OLS") is a widely-used ontology search engine created and hosted by EMBL-EBI. It is provided at no cost to the worldwide scientific community. At the time of writing (October 2019), it incorporates and indexes fully 239 separate ontologies, primarily from the realm of biomedical sciences. The OLS search engine is exposed both through its web front-end, and through a flexible REST API.

Behind the scenes, because the OLS software includes a generic indexer, for ontologies which can be ingested in either OBO or OWL format, OLS is highly extensible in that it can be adapted to cover the specialised data requirements of a particular group, company or institute. A research group might wish to have a firewalled version of OLS, in order to index one or more of its own ontologies absent from the public release, e.g. because the ontology might be pre-release, or might contain confidential data.

### Setting up an instance on Embassy Cloud ###

Before you can run OLS on the Embassy Cloud, you need to set up an instance (i.e. a virtual machine or VM). Possibly you have already done so; maybe this very Jupyter notebook is being served from such an instance. Detailed instructions on how to set up an instance are provided [here](https://www.ebi.ac.uk/seqdb/confluence/display/EMBCLOUD/Getting+started+in+OpenStack "Getting started with Embassy Cloud OpenStack"), but note that this applies to a generic, default configuration. For the specific purpose of setting your instance up to host a private version of OLS, we enumerate the minimally necessary steps below:

1. Once you have been sent your Embassy Cloud authentication details, go to the relevant URL—e.g. [http://extcloud05.ebi.ac.uk](http://extcloud05.ebi.ac.uk "Embassy Cloud login page")—and log in. You should see your dashboard, which defaults to "Overview" (at the top of which is a set of metrics depicted as pie charts).  
&nbsp;
2. You need an SSH key pair to enable password-less login from your local machine's (e.g. your laptop's) shell. From the Embassy Cloud dashboard, click on "Access & Security", then on "Key Pairs", then on "Create Key Pair". You get to choose a name for your new key pair, e.g. "ols_embassy", whereupon your browser will download a new private key to your local machine (your public key gets copied to your instance—see below); save this to your ssh key directory, typically `~/.ssh/`  
&nbsp;
3. From within the shell on your desktop or laptop, change the access rights to your new private key:  
```chmod u:rw,g:,o: ./.ssh/ols_embassy.pem```  
&nbsp;
4. Returning to your Embassy Cloud dashboard, you now need to configure access to various applications on your OLS instance. In the background, OLS runs versions of MongoDB and of Solr, both of which you can optionally expose on their respective ports. OLS itself runs within a Tomcat web container, accessible by default on port 8080. So you typically want to expose these three ports, plus the SSH port for console access. You can do this from the dashboard by creating a "Security Group". Click on "Access & Security", then on "Security Groups", then on "Create Security Group", then give your security group a name, e.g. "ols_runner", with an optional description. Once created, your new security group will appear among the pre-exisitng ones. Press the button to its right, labelled "Manage Rules".  
&nbsp;
5. You should now find yourself in a new screen, at the top of which you will see a button "Add Rule"; press this to activate the "Add Rule" dialogue. You need to add four new rules as follows.
    + Firstly, in the "Rule" dropdown, choose "SSH", leaving the other fields unchanged; click "Add" at the bottom  right of the dialogue; SSH will be exposed on its default port number 22.
    + Secondly, expose the MongoDB port: under "Rule", choose "Custom TCP Rule" and in the blank text box under "Port", enter "27017" and click "Add".
    + Thirdly, expose the Solr port: under "Rule", choose "Custom TCP Rule" and in the blank text box under "Port", enter "8983" and click "Add".
    + Finally, you must expose Tomcat: under "Rule", choose "Custom TCP Rule" and in the blank text box under "Port", enter "8080" and click "Add".  
&nbsp;
6. Next, in your dashboard, click on the "Images" sub-tab. Each image represents a pre-provisioned, ready-configured Linux operating system upon which you can build your initial running instance. At the bottom of the list of images you should see "Ubuntu18.04LTS", which is what you will use to run OLS in the Embassy Cloud. Click the "Launch" button to its right.  
&nbsp;
7. You should now be presented with the "Launch Instance" dialogue: call this instance "ols-embassy" or something equally meaningful and obvious; accept the default availability zone of "nova" and leave the instance count set to 1. Click "Next" towards the bottom right of the dialogue.  
&nbsp;
8. The sidebar should indicate that you are now in the "Source" section of the dialogue. Under "Select Boot Source" you should see "Image" pre-selected; leave this as it is. To its right however, change "Create New Volume" from "No" to "Yes". Accept the default device name "vda", a text box for which now appears underneath. Just below that, under "Allocated", you should see your selected image named "Ubuntu18.04LTS"; if so, this is correct, and you can proceed to the next screen. Click "Next".  
&nbsp;
9. You should find yourself in the "Flavour" section of the dialogue. A reasonable base configuration for OLS is the "s1.medium" flavour, which you should see in the list, about fifth from the top. Select this flavour using the "+" button to its right, and then click "Next".  
&nbsp;
10. You will find yourself in the "Networks" section of the dialogue. Probably, you will have at one network showing as available, which has been pre-created for you, with an attached router. Select your preferred network using the "+" button to its right; then click "Next".  
&nbsp;
11. You should be in the "Network Ports" section. You don't need to choose anything here. Click "Next".  
&nbsp;
12. You are now in the "Security Groups" section. Here you need to attach your instance to the security group that you created earlier, as this will allow web or console access on the various required ports. Select your security group, e.g. "ols_runner", and select it using the "+" button to its right. Click "Next".  
&nbsp;
13. You are now in the "Key Pair" section. You want to select your existing key pair, which should appear in the "Available" list below. Select it by pressing the "+" boutton to its right; then click "Next".  
&nbsp;
14. For the time being you can ignore the two bottom-most sections, "Configuration" and "Metadata". You should be ready to run your instance. Click the "Launch Instance" button on the bottom right.  
&nbsp;
15. You now need to associate an external-facing IP address so that you can access your instance over the internet. From within the "Instances" sub-tab, you should be able to see that the "Power State" of your instance is "Running". Click the arrow in the "More Actions" dropdown to its right, ignoring the default selection (e.g. "Create Snapshot"); instead, select "Associate Floating IP". Because no floating IP is assigned, you can request one by clicking "+". You'll then be asked to select from a "Pool". Not all of the listed pools are likely to work; you will be probably have been sent information about your pool along with the email containing the details of your Embassy OpenStack account: choose that pool. You will be shown your allotted IP address, along with a port number (don't worry about the port number). Both the new floating IP address and the pre-existing internal (subnet) address should be now visible in the list of instances.  
&nbsp;
16. Bearing in mind the settings of the security group which you set up and attached earlier, your instance should be open on port 22, the default SSH port. To set up a running, customised version of OLS on your instance, you need first to open a shell on it by SSHing in. From the shell on your laptop or desktop:
    
    **```bash#```**```ssh -i ~/.ssh/ols_embassy.pem -l ubuntu 193.62.```*```my.ip```*  
&nbsp;  
You should now be at the bash prompt **`ubuntu@ols-embassy:~$`** of your new instance ("ubuntu" is the default user name).  
&nbsp;
17. You need to update your Ubuntu installation in preparation for compiling and then containierising OLS. Execute the following commands.
    
    **```ubuntu@ols-embassy:~$```**```sudo apt update```  
    **```ubuntu@ols-embassy:~$```**```sudo apt upgrade```  
&nbsp;  
(Agree when prompted to install a new version of grub init in repsonse to the prompt—it only involves some whitespace changes anyway.)
    
    **```ubuntu@ols-embassy:~$```**```sudo apt autoremove```  
&nbsp;  
Having completed these steps, soft-reboot the instance, which you can do from the "More Actions" dropdown on your Embassy OpenStack dashboard.  
&nbsp;
18. Log back in; then install basic dependencies and utilities, including the MongoDB client:
    
    **```bash#```**```ssh -i ~/.ssh/ols_embassy.pem -l ubuntu 193.62.```*```my.ip```*  
    **```ubuntu@ols-embassy:~$```**```sudo apt install apt-transport-https gnupg-agent emacs-nox lynx mongodb-clients```  
&nbsp;  
19. Install the latest version of Docker, from Docker's own package repository:
    
    **```ubuntu@ols-embassy:~$```**```curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -```  
&nbsp;  
    (The host should report "OK".)
    
    **```ubuntu@ols-embassy:~$```**```sudo apt-key fingerprint 0EBFCD88```  
    **```ubuntu@ols-embassy:~$```**```sudo add-apt-repository \
      "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"```  
    **```ubuntu@ols-embassy:~$```**```sudo apt install docker-ce docker-ce-cli containerd.io```  
    **```ubuntu@ols-embassy:~$```**```sudo usermod -aG docker ubuntu```  
    **```ubuntu@ols-embassy:~$```**```newgrp docker```  
&nbsp;  
20. You are now ready to clone from the OLS-docker repository. After this, you can edit the YAML configuration file, to specify which ontologies should be loaded in your custom OLS-in-Docker build.
    
    **```ubuntu@ols-embassy:~$```**```git clone https://github.com/EBISPOT/OLS-docker.git```  
    **```ubuntu@ols-embassy:~$```**```cd OLS-docker```  
    **```ubuntu@ols-embassy:~$```**```emacs ols-config.yaml```  
&nbsp;  
    (Regarding the above, you may of course use any editor of your choice; just be aware that you might have to install it explicitly, in the stead of emacs—see point 18.)  
&nbsp;  
21. Build your OLS docker image and fire up a container.
    
    **```ubuntu@ols-embassy:~$```**```docker build -t ols .```  
    **```ubuntu@ols-embassy:~$```**```docker run -d -p 8080:8080 -t ols```  
&nbsp;  
22. Give your brand new instance of OLS a minute or so to come up fully. Then, if you have followed the instructions above, and things have gone smoothly, you should be able to see a running, cutomised OLS of your own, running on your own cloud instance. Check this by entering your Embassy IP address (193.62.*my.ip*) into your browser's address bar. Note that correctly mapping ports between Docker and Embassy OpenStack, it is entirely possible to run multiple, differently-configured instances of OLS in parallel, through multiple ports on a single Embassy IP address ... but we leave this topic for a later date.  
&nbsp;  
23. Assuming your browser can see your Embassy OLS instance ok, you should be ready to test its REST API. We'll now look at some examples in Python.

In [ ]:
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import requests, json, matplotlib.pyplot as plt, pandas as pd, ipywidgets as widgets

# List loaded ontologies
r1 = requests.get('http://193.62.54.138:8080/api/ontologies')
# print(r.content)
# print(r1.json())
# print(json.dumps(r1.json(), indent=2))

# List terms in an ontology
r2 = requests.get('http://193.62.54.138:8080/api/ontologies/efo/terms')
# print(json.dumps(r2.json(), indent=2))

# Search for a query term
r3 = requests.get('http://193.62.54.138:8080/api/select?q=diabetes')
# print(json.dumps(r3.json(), indent=2))

# Search for a query term in a particular ontology (or set of ontologies)
text4 = widgets.Text(description='Enter string:', placeholder='Type OLS search term(s) here')
button4 = widgets.Button(description='Search OLS')
output4 = widgets.Output(layout={'border': '2px solid red', 'padding': '20px'})
display(text4, button4)
display(output4)
with output4:
    def on_button_clicked(b):
        # output4 = widgets.Output()
        # display(text4, button4, output4)
        # print("Clickety-clack!")
        display(text4)
        print(text4.value)
        params4 = dict(q=text4.value, ontology="efo")
        r4 = requests.get('http://193.62.54.138:8080/api/select', params=params4)
        print(json.dumps(r4.json(), indent=2))
button4.on_click(on_button_clicked)

# params4 = dict(q="diabetes", ontology="efo")
## r4 = requests.get('http://193.62.54.138:8080/api/select?q=diabetes&ontology=efo')
# r4 = requests.get('http://193.62.54.138:8080/api/select', params=params4)
# print(json.dumps(r4.json(), indent=2))

# plt.plot([[0,0],[1,1]], linewidth=3, label='stochastical diagonal')
# plt.show()
# plt.bar([1,2,3,4,5,6], [20,24,18,16,29,14])
# plt.show()